In [3]:
import pandas as pd
import numpy as np


In [4]:
df= pd.read_csv('/content/drive/My Drive/train.csv')

In [7]:
df.shape, df.head(), df.tail()

((42000, 785),
    label  pixel0  pixel1  pixel2  ...  pixel780  pixel781  pixel782  pixel783
 0      1       0       0       0  ...         0         0         0         0
 1      0       0       0       0  ...         0         0         0         0
 2      1       0       0       0  ...         0         0         0         0
 3      4       0       0       0  ...         0         0         0         0
 4      0       0       0       0  ...         0         0         0         0
 
 [5 rows x 785 columns],
        label  pixel0  pixel1  pixel2  ...  pixel780  pixel781  pixel782  pixel783
 41995      0       0       0       0  ...         0         0         0         0
 41996      1       0       0       0  ...         0         0         0         0
 41997      7       0       0       0  ...         0         0         0         0
 41998      6       0       0       0  ...         0         0         0         0
 41999      9       0       0       0  ...         0         0       

In [8]:
X= df.iloc[:,1:].values
y= df.iloc[:,0].values

In [10]:
X.shape,X

((42000, 784), array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]))

In [12]:
y.shape,y, np.unique(y)


((42000,),
 array([1, 0, 1, ..., 7, 6, 9]),
 array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))

* Here I performed the train_test_split without using any external libraries

In [15]:
train_length = len(X)*0.75
train_length

31500.0

In [23]:
train_X,test_X = X[:int(train_length),:],X[int(train_length):,:]
train_y,test_y=y[:int(train_length)], y[int(train_length):]


In [24]:
train_X.shape, test_X.shape, train_y.shape, test_y.shape

((31500, 784), (10500, 784), (31500,), (10500,))

* Reshaping the data into required format so that it can trained in a neural net.

In [26]:
train_X = train_X.reshape(train_X.shape[0],28,28,1)
test_X = test_X.reshape(test_X.shape[0],28,28,1)

train_X.shape, test_X.shape

((31500, 28, 28, 1), (10500, 28, 28, 1))

* Normalizing the data

In [27]:
train_X= train_X/255
test_X= test_X/255

* One hot encoding to the dependent variable


In [28]:
import keras 

train_y= keras.utils.to_categorical(train_y,10)
test_y= keras.utils.to_categorical(test_y,10)


Using TensorFlow backend.


# CNN Model

In [35]:
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Dropout, BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

In [39]:
model = Sequential()

model.add(Conv2D(64,(3,3), activation='relu',input_shape=(28,28,1)))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))


model.add(Conv2D(128,(3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))


model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(84,activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(10,activation='softmax'))

In [40]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [41]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 128)       73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 128)         0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 5, 5, 128)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 3200)             

In [49]:
class myCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch,logs={}):
    if(logs.get('loss')<0.05):
      print('Accuracy is more than 99%')
      self.model.stop_training=True

callbacks=myCallback()

In [50]:
model.fit(train_X, train_y, batch_size=16, epochs= 50, verbose=1, validation_data=(test_X,test_y),callbacks=[callbacks])


Train on 31500 samples, validate on 10500 samples
Epoch 1/50
31500/31500 [==============================] - 78s 2ms/step - loss: 0.0404 - accuracy: 0.9884 - val_loss: 0.0345 - val_accuracy: 0.9899
Accuracy is more than 99%


In [51]:
score= model.evaluate(test_X,test_y)
print('loss',score[0])
print('accuracy',score[1])

10500/10500 [==============================] - 5s 457us/step
loss 0.03451149111186498
accuracy 0.9899047613143921


In [52]:
test = pd.read_csv('/content/drive/My Drive/test.csv')

In [53]:
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
test= test.values.reshape(test.shape[0],28,28,1)


In [57]:
test.shape

(28000, 28, 28, 1)

* predicting for the test data

In [61]:
predicted=model.predict_classes(test)

In [62]:
predicted[0]

2

In [63]:
sub=pd.read_csv('/content/drive/My Drive/sample_submission.csv')

In [64]:
sub.head()

,ImageId,Label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [67]:
sub['Label']= predicted

In [74]:
sub.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3


In [75]:
sub.to_csv('submission.csv',index=False)